In [1]:
from obspy import read
from obspy.signal.trigger import carl_sta_trig
from obspy.signal.trigger import recursive_sta_lta
from obspy.signal.trigger import ar_pick
from obspy.signal.trigger import z_detect
from obspy.signal.trigger import trigger_onset
from obspy.signal.trigger import plot_trigger
from obspy.signal.trigger import coincidence_trigger
from obspy.signal.trigger import coincidence_trigger, plot_trigger, trigger_onset
import numpy as np
import obspy
import os
from obspy.signal.trigger import z_detect



from obspy.core import read
from obspy.signal.trigger import plot_trigger
st = read("")
st = st.select(channel="HHZ")
st 
print(st)  
print(st[0].stats) 


c:\Python311\Lib\site-packages\pkg_resources\__init__.py:123: PkgResourcesDeprecationWarning: p is an invalid version and will not be supported in a future release
  warnings.warn(


FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
x = len(st)
i=0
for i in range(x):
    trace = read("")[i]
    trace = st[i]
    df = trace.stats.sampling_rate
    from obspy.signal.trigger import classic_sta_lta
    cft = z_detect(trace.data, int(0.5 * df))
    plot_trigger(trace, cft, 9, 0.5)

In [ ]:


# Path to the directory containing the mseed files
mseed_dir = ""

# Define the trigger threshold
threshold = 5.0  # Example threshold value, adjust as needed

# Loop through each mseed file in the directory
for filename in os.listdir(mseed_dir):
    if filename.endswith(".mseed"):
        # Read in the mseed file
        st = obspy.read(os.path.join(mseed_dir, filename))

        # Counter for CSV files
        event_counter = 0

        # Iterate over each event in the file
        for i, event in enumerate(st):
            if event.stats.channel == "HHZ":  # Filter only the HHZ channel
                # Create a new CSV file for the event
                csv_filename = f"{os.path.splitext(filename)[0]}_event{event_counter}.csv"
                event_counter += 1
                # Extract metadata from the event
                station = event.stats.station
                channel = event.stats.channel
                starttime = event.stats.starttime

                # Calculate the Z-score of the event data
                df = event.stats.sampling_rate
                z_scores = z_detect(event.data, int(0.5 * df))

                # Find indices where the Z-score values exceed the threshold
                trigger_indices = np.where(z_scores > threshold)[0]

                # Find the first trigger onset index and last trigger off index
                if len(trigger_indices) > 0:
                    first_onset_index = trigger_indices[0]
                    first_onset_time = starttime + first_onset_index / event.stats.sampling_rate

                    last_off_index = trigger_indices[-1]
                    last_off_time = starttime + last_off_index / event.stats.sampling_rate

                    # Print the first trigger onset index and last trigger off index for each event
                    print(f"Event {i+1}, Channel {channel}:")
                    print(f"First Trigger onset index: {first_onset_index}, Time: {first_onset_time}")
                    print(f"Last Trigger off index: {last_off_index}, Time: {last_off_time}")

                    # Write the event data to the CSV file with metadata as a header
                    with open(csv_filename, "w") as f:
                        f.write("Timestamp, Value, label\n")
                        for t, v in zip(event.times(), event.data):
                            timestamp = (starttime + t).strftime("%Y-%m-%dT%H:%M:%S.%fZ")
                            label = 'quake' if first_onset_time <= starttime + t <= last_off_time else 'noise'
                            f.write(f"{timestamp}, {v}, {label}\n")

                else:
                    print(f"No triggers found for Event {i+1}, Channel {channel}.")
